# Autoencoder cf study #1
<hr style='height:5px'>
kwanki.ahn<br>
2019/06/01<br>
ref : 
<hr style='height:5px'>

In [25]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [60]:
def get_users_items_ratings_nparray():
    # movielens dataset : https://old.datahub.io/dataset/movielens
    U_PATH = 'dataset/ml-1m/users.dat'
    M_PATH = 'dataset/ml-1m/movies.dat'
    R_PATH = 'dataset/ml-1m/ratings.dat'
    
    # data processing : https://github.com/khanhnamle1994/movielens/blob/master/Data_Processing.ipynb
    # Specify User's Age and Occupation Column
    AGES = { 1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44", 45: "45-49", 50: "50-55", 56: "56+" }
    OCCUPATIONS = { 0: "other or not specified", 1: "academic/educator", 2: "artist", 3: "clerical/admin",
                    4: "college/grad student", 5: "customer service", 6: "doctor/health care",
                    7: "executive/managerial", 8: "farmer", 9: "homemaker", 10: "K-12 student", 11: "lawyer",
                    12: "programmer", 13: "retired", 14: "sales/marketing", 15: "scientist", 16: "self-employed",
                    17: "technician/engineer", 18: "tradesman/craftsman", 19: "unemployed", 20: "writer" }
    
    # Read the Ratings File
    ratings = pd.read_csv(R_PATH, 
                        sep='::', 
                        engine='python', 
                        encoding='latin-1',
                        names=['user_id', 'movie_id', 'rating', 'timestamp'])

    # Set max_userid to the maximum user_id in the ratings
    max_userid = ratings['user_id'].drop_duplicates().max()
    # Set max_movieid to the maximum movie_id in the ratings
    max_movieid = ratings['movie_id'].drop_duplicates().max()

    # Process ratings dataframe for Keras Deep Learning model
    # Add user_emb_id column whose values == user_id - 1
    ratings['user_emb_id'] = ratings['user_id'] - 1
    # Add movie_emb_id column whose values == movie_id - 1
    ratings['movie_emb_id'] = ratings['movie_id'] - 1

    print len(ratings), 'ratings loaded'
    
    # Read the Users File
    users = pd.read_csv(U_PATH, 
                        sep='::', 
                        engine='python', 
                        encoding='latin-1',
                        names=['user_id', 'gender', 'age', 'occupation', 'zipcode'])
    users['age_desc'] = users['age'].apply(lambda x: AGES[x])
    users['occ_desc'] = users['occupation'].apply(lambda x: OCCUPATIONS[x])
    print len(users), 'descriptions of', max_userid, 'users loaded.'
    
    # Read the Movies File
    movies = pd.read_csv(M_PATH, 
                        sep='::', 
                        engine='python', 
                        encoding='latin-1',
                        names=['movie_id', 'title', 'genres'])
    print len(movies), 'descriptions of', max_movieid, 'movies loaded.'
    
    return users, movies, ratings

In [64]:
def check_stats(users, movies, ratings):
    n_user = len(users)
    n_item = len(movies)
    n_rating = len(ratings)
    n_matrix_size = n_user * n_item
    n_sparsity = n_rating / float(n_matrix_size) * 100
    print "users : {}, items : {}, ratings : {}, sparsity : {}".format(n_user, n_item, n_rating, n_sparsity)
    print "user_id max : {}, movie_id max : {}".format(users['user_id'].max(), movies['movie_id'].max())

In [67]:
def build_dict_of_data(users, movies, ratings):
    user2idx = {}
    idx2user = {}
    movie2idx = {}
    idx2movie = {}

    for idx, row in users.iterrows():
        user2idx[row['user_id']] = idx
        idx2user[idx] = row['user_id']
    for idx, row in movies.iterrows():
        movie2idx[row['movie_id']] = idx
        idx2movie[idx] = row['movie_id']
    return user2idx, idx2user, movie2idx, idx2movie

In [71]:
def build_ratings_matrix(ratings, user2idx, movie2idx):
    r_matrix = np.zeros((n_user, n_item), dtype='i')
    for index, row in ratings.iterrows():
        user_idx = user2idx[row['user_id']]
        movie_idx = movie2idx[row['movie_id']]
        rating = row['rating']
        r_matrix[user_idx][movie_idx] = rating
    return r_matrix

In [334]:
from IPython.display import Image, SVG
import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras import regularizers

def build_autoencoder_model(x_train, latent_size = 32):
    # input dimension = 784
    input_dim = x_train.shape[1]
    inter_dim = 1000
    rep_dim = 50  # latent_size  # 32

    compression_factor = float(input_dim) / rep_dim
    print("Compression factor: %s" % compression_factor)

    autoencoder = Sequential()
    autoencoder.add(
        Dense(inter_dim, input_shape=(input_dim,), activation='relu')
    )
    autoencoder.add(
        Dense(rep_dim, activation='relu')
    )
    autoencoder.add(
        Dense(inter_dim, activation='relu')
    )
    autoencoder.add(
        Dense(input_dim, activation='relu')#'sigmoid')
    )

    autoencoder.summary()
    
    return autoencoder

In [297]:
def show_history_of_user(ratings, movies, sample_user_id=100, item_number=15):
    print "== get_history_of_user(), user id = {}".format(sample_user_id)
    history_with_id_rating = (ratings.loc[ratings['user_id'] == sample_user_id][['movie_id', 'rating']]
            .sort_values(["rating"], ascending=[False])[:item_number])
    history_with_meta = pd.merge(history_with_id_rating, movies, on='movie_id')
    return history_with_meta

In [299]:
def get_recommend_of_user_raw(model, r_matrix, user2idx, movies, sample_user_id=100):
    print "== get_recommend_of_user_raw(), user id = {}".format(sample_user_id)
    # reshape 1 by x => x by 1
    history_of_user = r_matrix[user2idx[sample_user_id]]
    shape_len = history_of_user.shape[0]
    history_of_user = history_of_user.reshape((1, shape_len))
    score_list = model.predict(history_of_user).flatten()
    print "  result raw : shape = {}, max = {}, min = {}".format(score_list.shape, score_list.max(), score_list.min())
    return score_list

In [300]:
def get_watched_remove(recommends_raw, r_matrix, user2idx, idx2movie, sample_user_id=100):
    print "== get_watched_remove() =="
    
    # remove watched
    rating_of_user = r_matrix[user2idx[sample_user_id]]
    print rating_of_user.shape

    recommends_watch_removed = []
    for idx in range(0, len(recommends_raw)):
        if rating_of_user[idx] == 0:  # FALSE == if user watched idx program already
            recommends_watch_removed.append((idx2movie[idx], recommends_raw[idx]))
            
    return recommends_watch_removed

In [311]:
def show_recommend_of_user(model, r_matrix, user2idx, idx2movie, movies, sample_user_id=100, item_number=15):
    print "== get_recommend_of_user(), user id : {} ==".format(sample_user_id)
    # model's output (raw) : list of scores, length is item count, from index 0 (idx2movie needs)
    recommends_raw = get_recommend_of_user_raw(model, r_matrix, user2idx, movies, sample_user_id)
    
    # remove watched (from rating matrix, allow only if not rating score is 0) -> [(movie id, score)]
    remove_watched = get_watched_remove(recommends_raw, r_matrix, user2idx, idx2movie, sample_user_id)
    
    # select top n
    from operator import itemgetter
    remove_watched.sort(key=itemgetter(1), reverse=True)
    remove_watched = remove_watched[:item_number]

    recommend_df = pd.DataFrame(np.array(remove_watched))
    recommend_df.columns = ['movie_id', 'inferred_score']
    recommend_df = recommend_df.astype({"movie_id": int, "inferred_score": float})
    recommend_with_meta = pd.merge(recommend_df, movies, on='movie_id')
    return recommend_with_meta

# MAIN

## DATA

In [317]:
users, movies, ratings = get_users_items_ratings_nparray()

218508 ratings loaded
6040 descriptions of 1325 users loaded.
3883 descriptions of 3952 movies loaded.


In [318]:
ratings[:10]

,user_id,movie_id,rating,timestamp,user_emb_id,movie_emb_id
0,1,1193,5,978300760,0,1192
1,1,661,3,978302109,0,660
2,1,914,3,978301968,0,913
3,1,3408,4,978300275,0,3407
4,1,2355,5,978824291,0,2354
5,1,1197,3,978302268,0,1196
6,1,1287,5,978302039,0,1286
7,1,2804,5,978300719,0,2803
8,1,594,4,978302268,0,593
9,1,919,4,978301368,0,918


In [319]:
users[:10]

,user_id,gender,age,occupation,zipcode,age_desc,occ_desc
0,1,F,1,10,48067,Under 18,K-12 student
1,2,M,56,16,70072,56+,self-employed
2,3,M,25,15,55117,25-34,scientist
3,4,M,45,7,02460,45-49,executive/managerial
4,5,M,25,20,55455,25-34,writer
5,6,F,50,9,55117,50-55,homemaker
6,7,M,35,1,06810,35-44,academic/educator
7,8,M,25,12,11413,25-34,programmer
8,9,M,25,17,61614,25-34,technician/engineer
9,10,F,35,1,95370,35-44,academic/educator


In [320]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [339]:
check_stats(users, movies, ratings)
user2idx, idx2user, movie2idx, idx2movie = build_dict_of_data(users, movies, ratings)
len(user2idx), len(idx2user), len(movie2idx), len(idx2movie)

users : 6040, items : 3883, ratings : 218508, sparsity : 0.931671933867
user_id max : 6040, movie_id max : 3952


(6040, 6040, 3883, 3883)

In [323]:
r_matrix = build_ratings_matrix(ratings, user2idx, movie2idx)

## MODEL

In [329]:
autoencoder = build_autoencoder_model(r_matrix, latent_size = 150)

Compression factor: 25.8866666667
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 150)               582600    
_________________________________________________________________
dense_13 (Dense)             (None, 3883)              586333    
Total params: 1,168,933
Trainable params: 1,168,933
Non-trainable params: 0
_________________________________________________________________


In [335]:
autoencoder = build_autoencoder_model(r_matrix, latent_size = 150)

Compression factor: 77.66
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 1000)              3884000   
_________________________________________________________________
dense_15 (Dense)             (None, 50)                50050     
_________________________________________________________________
dense_16 (Dense)             (None, 1000)              51000     
_________________________________________________________________
dense_17 (Dense)             (None, 3883)              3886883   
Total params: 7,871,933
Trainable params: 7,871,933
Non-trainable params: 0
_________________________________________________________________


In [336]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(r_matrix, r_matrix,
                epochs=30,
                batch_size=256,
                shuffle=True
                # validation_data=(x_test, x_test)
               )

Epoch 1/30
6040/6040 [==============================] - 6s - loss: 0.1050     
Epoch 2/30
6040/6040 [==============================] - 5s - loss: 0.0925     
Epoch 3/30
6040/6040 [==============================] - 5s - loss: 0.0863     
Epoch 4/30
6040/6040 [==============================] - 5s - loss: 0.0819     
Epoch 5/30
6040/6040 [==============================] - 5s - loss: 0.0781     
Epoch 6/30
6040/6040 [==============================] - 5s - loss: 0.0748     
Epoch 7/30
6040/6040 [==============================] - 5s - loss: 0.0688     
Epoch 9/30
6040/6040 [==============================] - 5s - loss: 0.0659     
Epoch 10/30
6040/6040 [==============================] - 5s - loss: 0.0631     
Epoch 11/30
6040/6040 [==============================] - 5s - loss: 0.0605     
Epoch 12/30
6040/6040 [==============================] - 5s - loss: 0.0580     
Epoch 13/30
6040/6040 [==============================] - 5s - loss: 0.0554     
Epoch 14/30
6040/6040 [=========================

## VERIFY

In [346]:
show_history_of_user(ratings, movies, sample_user_id=45)

== get_history_of_user(), user id = 45


,movie_id,rating,title,genres
0,915,5,Sabrina (1954),Comedy|Romance
1,3044,5,Dead Again (1991),Mystery|Romance|Thriller
2,969,5,"African Queen, The (1951)",Action|Adventure|Romance|War
3,46,5,How to Make an American Quilt (1995),Drama|Romance
4,357,5,Four Weddings and a Funeral (1994),Comedy|Romance
5,440,5,Dave (1993),Comedy|Romance
6,1835,5,City of Angels (1998),Romance
7,2716,5,Ghostbusters (1984),Comedy|Horror
8,2779,5,Heaven Can Wait (1978),Comedy
9,587,5,Ghost (1990),Comedy|Romance|Thriller


In [347]:
show_recommend_of_user(autoencoder, r_matrix, user2idx, idx2movie, movies, sample_user_id=45)

== get_recommend_of_user(), user id : 45 ==
== get_recommend_of_user_raw(), user id = 45
  result raw : shape = (3883,), max = 5.79262495041, min = 0.0
== get_watched_remove() ==
(3883,)


,movie_id,inferred_score,title,genres
0,2080,2.064404,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
1,480,1.457573,Jurassic Park (1993),Action|Adventure|Sci-Fi
2,2948,1.381002,From Russia with Love (1963),Action
3,356,1.371155,Forrest Gump (1994),Comedy|Romance|War
4,21,1.298706,Get Shorty (1995),Action|Comedy|Drama
5,110,1.255246,Braveheart (1995),Action|Drama|War
6,1210,1.074322,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
7,3210,0.994492,Fast Times at Ridgemont High (1982),Comedy
8,1777,0.987137,"Wedding Singer, The (1998)",Comedy|Romance
9,3510,0.978562,Frequency (2000),Drama|Thriller
